# Python Demo For All-GRAD Meetup 

#### Michael Smith,  May, 2016

## Intro

This notebook is to serve as a demonstration of a real use-case for Python and some of it's common data science packages.  In helping prepare analysis to be used in Nielsen's presentation for the POI Conference, I leveraged Python to do some of the repeatable analysis so that I could produce results quickly and in an easily repeatable method.  The class I built is called 'benchmark' and it includes the following functions:

    - read_data:  Takes a csv file and converts it to a pandas dataframe
    - find_roi:  Caluclates ROI given an aggregation classifier and a year
    - find_coeff:  Prepares requested coefficients from Norms models
    - coeff_roi_reg:  Plots ROI vs a requested norms coefficient and the trendline and gives the regression equation and R^2
    - elast_clustering:  Runs K-Means analysis for a given Norms coefficient against ROI for n clusters (n : [1,6]) 
        and gives both the plot of resulting clusters and returns the labeling in a dataframe

## Using an IPython Notebook:
This notebook is made up of cells.  There are two types within this particular notebook:  markdown (like this one) and code (those with "In []" next to them).  To run a cell of code, click inside that cell and press Shift+Enter.  This runs the cell and places you within the next cell.

## Package selection
First we pull in the required packages for handling datasets and analyzing data

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
import matplotlib.pyplot as plt
import bokeh.io
import bokeh.mpl
import bokeh.plotting
from sklearn.cluster import KMeans
bokeh.io.output_notebook()

%install_ext https://raw.githubusercontent.com/rasbt/watermark/master/watermark.py
%load_ext watermark

## Notebook Options
These are specific for the IPython (Jupyter) notebook environment.

The first, '%matplotlib inline', dictates that any plots will be rendered inline with the cells in this notebook.

The second is a package, '%watermark' which keeps track of versions of Python and packages being used within this notebook.  It is helpful and good practice to track this information for reproducibility.


In [ ]:
%matplotlib inline
%watermark -v -m -p pandas,numpy,statsmodels,matplotlib

## Creating the Benchmarks class.
Below is the major chunk of code where I define the class and functions within it.  You can see the basic structure very easily by following the indention structure.

    Class
        function 1(options):
            code
        
        ...
        
        function n(options):
            code

Running this cell of code will 'initialize' the class and make it available throughout the rest of the python session.

In [ ]:
class benchmark(object):
    """Class for doing some analysis for POI Conference Presentation"""
    
    def read_data(csv_path):
        """Reads in data which is already merged in the form
           Benchmark data aggregated by country,year,category,roi range followed by
           model coefficients from Core Models"""
        
        try:
            data = pd.read_csv(csv_path)
            data.columns = ['category', 'year','market_channel', 'department',
                'roi_range', 'value', 'baseline_value', 'trade_spend_total',
                'trade_spend_incl_cogs', 'number_of_tpr_weeks',
                'number_of_display_weeks', 'number_of_feature_weeks',
                'number_of_event_weeks', 'mfr_roi_value', 'trade_direct_promo_spend',
                'number_of_weeks','number_of_records', 'PPI', 'BP', 'FEDI', 'DISO',
                'FEAO']
            
        except ValueError:
            print('File is either unavailable or non-existent.')
        
        return data

    def find_roi(in_data,agg_var='category',year=2015):
        """Calculates ROI for a Benchmark dataset given a level of aggregation and year"""
        
        roi_data = in_data[[agg_var,'year','roi_range','mfr_roi_value','trade_direct_promo_spend']]
        roi_data = roi_data[(roi_data.year==year) & (roi_data.roi_range.notnull())]
        roi_pivot = pd.pivot_table(roi_data,values=['mfr_roi_value','trade_direct_promo_spend'], 
                                   index=[agg_var],aggfunc='sum')

        roi_pivot['roi'] = (roi_pivot['mfr_roi_value']/roi_pivot['trade_direct_promo_spend']) + 1
        
        return roi_pivot.drop(roi_pivot.columns[[0,1]],axis=1)
        
    def find_coeff(in_data,agg_var='category',coeff='PPI'):
        """Compiles requested Norms coefficient given an aggregation level"""
        
        return pd.pivot_table(in_data,values=[coeff],index=[agg_var],aggfunc='mean')
        
    def graph(formula,x_range):
        """Helper function for graphing a line equation"""
        
        x = np.array(x_range)
        y = eval(formula)
        plt.plot(x,y)
        
    def coeff_roi_reg(in_data,agg_var,year,norms_coeff,path='none',interactive='False'):
        """Plots ROI vs given Norms coefficient and plots the regression line.  Output is a plot
           with the line, the equation, and the R-Squared included"""
        
        coeff_roi_data = pd.merge(benchmark.find_roi(in_data,agg_var,year),
                                  benchmark.find_coeff(in_data,agg_var,norms_coeff),
                                  left_index=True, right_index=True)
        coeff_roi_data = coeff_roi_data[coeff_roi_data['roi'].notnull() & coeff_roi_data[norms_coeff].notnull()]
        
        roi = coeff_roi_data['roi']
        
        coeff = sm.add_constant(coeff_roi_data[norms_coeff])
        
        model = sm.OLS(roi,coeff)
        results = model.fit()
        #find equation for graph
        intercept = round(results.params[0],3)
        coeff_effect = round(results.params[1],3)
        trendline = str(intercept)+'+x*'+str(coeff_effect)
        r2 = round( results.rsquared, 3)
        
        plt.plot(coeff[norms_coeff],roi,'o')
        
        benchmark.graph(trendline,coeff[norms_coeff])
        
        plt.xlabel(norms_coeff)
        plt.ylabel('ROI')
        plt.title('Promo ROI vs ' + norms_coeff + ' by ' + agg_var + ' - ' + str(year))        
        plt.text(min(coeff[norms_coeff]),max(roi)-.15,'ROI ~ ' + str(intercept) + str(coeff_effect) + '*' + norms_coeff 
                 + '\n' + 'R-Squared: '+str(r2))
        if(path=='none'):
            if(interactive):
                bokeh.plotting.show(bokeh.mpl.to_bokeh())
            else:
                plt.show()
        else:
            plt.savefig(path)
    
    def elast_clustering(in_data,norms_coeff,year,clusters,interactive=False):
        """Runs K-Means analysis for a given Norms coefficient against ROI for n clusters (n : [1,6]) 
            and gives both the plot of resulting clusters and returns the labeling in a dataframe"""
        
        coeff_roi_data = pd.merge(benchmark.find_roi(in_data,'category',year),
                                  benchmark.find_coeff(in_data,'category',norms_coeff),
                                  left_index=True, right_index=True)
        coeff_roi_data = coeff_roi_data[coeff_roi_data['roi'].notnull() & coeff_roi_data[norms_coeff].notnull()]
        
        colors = ['#4EACC5', '#FF9C34', '#4E9A06','#E9461E','#8B008B','#808080']        
        
        kmeans = KMeans(n_clusters=clusters)
        kmeans.fit(coeff_roi_data)
        k_means_labels = kmeans.labels_
        k_means_cluster_centers = kmeans.cluster_centers_
        
        
        for k in range(clusters):
            my_members = k_means_labels == k
            cluster_center = k_means_cluster_centers[k]
            plt.plot(coeff_roi_data[norms_coeff][my_members],coeff_roi_data['roi'][my_members],'w',
                     markerfacecolor = 'none', markeredgecolor = colors[k], marker='o')
            plt.plot(cluster_center[1], cluster_center[0], 'o',
                      markerfacecolor=colors[k], markersize=8)
        
        plt.xlabel(norms_coeff)
        plt.ylabel('ROI')
        plt.title('K-Means Nearest Neighbors with ' + str(clusters) + ' Clusters.' + ' - ' + str(year))
        
        if(interactive):
            bokeh.plotting.show(bokeh.mpl.to_bokeh())
        else:
            plt.show()
        
        coeff_roi_data['label'] = k_means_labels
        
        return coeff_roi_data

## Using the Benchmark class
Now that we have these tools created, we can begin using them to produce analysis easily.

Let's first read in the sample dataset.  I created a method to read in the csv and name the columns from a less-intuitive NDX fact name to something that makes more sense.  

The data contains benchmark data aggregated to year, category, and range of event success (no event, lost money, broke even, made money).  It also contains category-aggregated coefficients from the Norms models.  

In [ ]:
data = benchmark.read_data('new_sample_in.csv')
data[1:6]

## Promotion Coefficients vs ROI
My initial questions were about the relationship between promotion ROI and various Core model coefficients.  Using my method 'coeff_roi_reg', I can easily specify the dataset to be used, whether I want to aggregate by department or by category, and which coefficient I want to compare to ROI. 

I'm able to make multiple calls easily over a list of strings in a loop to look for potential relationships.

In [ ]:
for coeff in ['PPI','BP','FEDI','FEAO','DISO']:
    benchmark.coeff_roi_reg(data,'category',2015,coeff,interactive=False)

## PPI vs ROI Over Time

Looking at the above, PPI (promoted price elasticity) seems to have a fairly strong relationship with ROI.  Next, I'm curious to see how that may be changing over time.  We can do this simply by looping over the available years in the data with a constant coefficient selection.

In [ ]:
for year in np.unique(data['year']):
    benchmark.coeff_roi_reg(data,'category',year,'PPI',interactive=True)    

We can see that the relationship gets slightly weaker over time and the effect dampens slightly as well.

## Elasticity Clustering

With this knowledge of an existing relationship between PPI and ROI, I wanted to see if I could break the relationship into meaninful clusters of categories.  For example:  "High Return, High Elasticity" vs "Low Return, Low Elasticity".  To do this, I use the K-Means module from the Sci-Kit Learn package within a module I created called 'elast_clustering'.  By simply calling the method, we get a plot of the nearest-neighbor clustering.  I loop through various cluster numbers.

In [ ]:
for n in range(1,6):
    benchmark.elast_clustering(data,'PPI',2015,n)

## Output Clustering Labels

The method, if assigned to a variable, will return a Dataframe of categories, their ROI, coefficient value, and the labeling output from the K-Means cluster assignments.  We could then create a .csv output file with the labels to share or use in further analysis. Pandas Dataframes have a method to do this called .to_csv().

In [ ]:
labeled_data = benchmark.elast_clustering(data,'PPI',2015,3,interactive=True)
print(labeled_data.sort_values('PPI'))

labeled_data.to_csv('labeled_data_out.csv')